In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LassoCV, LinearRegression, Lasso
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

import sys
sys.path.append('../src/')
from ffcDataCleaning.CodeBook import getCodeDescription

In [ ]:
features = pd.read_csv('../output/imputing/.data/imputed.csv', index_col=0)
features.head()

In [ ]:
data_dir = '../.data'
fp_train = '{}/train.csv'.format(data_dir)
df_test = pd.read_csv(fp_train,index_col=0)

In [ ]:
Y = df_test.materialHardship.dropna()
print Y.shape
idx = Y.index

S = StandardScaler()
X = S.fit_transform(features.loc[idx])
print X.shape

In [ ]:
b = 500
reg = LassoCV(selection='random')
reg.fit(X[:b],Y[:b])

In [ ]:
from sklearn.feature_selection import SelectFromModel

ms = SelectFromModel(reg, threshold = 0.0001, prefit = True)
x_red = ms.transform(X)
impt_features = features.columns[ms.get_support()]

In [ ]:
print '\n'.join(['{:.<15s}{}'.format(x,getCodeDescription(x)) for x in impt_features])

In [ ]:
Y_pred = reg.predict(X[b:])
print mean_squared_error(Y[b:], Y_pred)

In [ ]:
reg2 = LassoCV()
reg2.fit(x_red,Y)
Y_pred = reg2.predict(x_red[b:])
print mean_squared_error(Y[b:], Y_pred)
print reg2.alpha_

In [ ]:
scores = cross_val_score(Lasso(alpha=.004,selection='random'),x_red,Y,cv=10,scoring='neg_mean_squared_error',n_jobs=-1)
print abs(scores)
print scores.mean()
